In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from typing import Any, Dict, List, Tuple
from pathlib import Path
import yaml

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandas_profiling import ProfileReport # profile report を作る用
from matplotlib_venn import venn2 # venn図を作成する用
from tqdm import tqdm
from contextlib import contextmanager
from time import time
from collections import Counter

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error, mean_squared_error
import lightgbm as lgb

from mykaggle.metric.mse import rmsle
from mykaggle.util.ml_logger import MLLogger
from mykaggle.lib.lgbm_util import compute_importances, save_importances
from mykaggle.lib.pandas_util import change_column_name
from mykaggle.util.routine import fix_seed

sns.set_style('darkgrid')

In [2]:
settings = yaml.safe_load('''
name: '188_eda'
competition: atmacup8
seed: 1019
training:
    num_folds: 5
    num_rounds: 1000
    early_stopping_rounds: 100
    verbose_eval: 20
lgbm_params:
    objective: binary
    learning_rate: 0.05
    max_depth: -1
    num_leaves: 31
    colsample_bytree: .7
    metric: "None"
''')

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
fix_seed(settings['seed'])

In [4]:
datadir = Path('../data/')
ckptdir = Path('../ckpt/') / settings['name']
if not ckptdir.exists():
    ckptdir.mkdir()

In [5]:
df_train = pd.read_csv(datadir / 'id_train.csv')
df_test = pd.read_csv(datadir / 'id_test.csv')
df_submission = pd.read_csv(datadir / 'atmaCup8_sample-submission.csv')
df_train.shape, df_test.shape

((8359, 17), (8360, 12))

In [6]:
df_train.head().T

,0,1,2,3,4
Name,LEGO Batman: The Videogame,LEGO Indiana Jones: The Original Adventures,LEGO Batman: The Videogame,Combat,LEGO Harry Potter: Years 5-7
Platform,Wii,Wii,PSP,2600,Wii
Year_of_Release,NaN,NaN,NaN,NaN,NaN
Genre,Action,Action,Action,Action,Action
Publisher,Warner Bros. Interactive Entertainment,LucasArts,Warner Bros. Interactive Entertainment,Atari,Warner Bros. Interactive Entertainment
NA_Sales,180,151,56,117,69
EU_Sales,97,61,44,7,42
JP_Sales,0,0,0,0,0
Other_Sales,28,21,27,1,12
Global_Sales,306,234,128,125,124


In [7]:
df_test.head().T

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4
Name,Hitman 2: Silent Assassin,Legacy of Kain: Soul Reaver,Metal Gear Solid 2: Substance,Silent Hill: Homecoming,Silent Hill: Homecoming
Platform,XB,PS,XB,X360,PS3
Year_of_Release,NaN,NaN,NaN,NaN,NaN
Genre,Action,Action,Action,Action,Action
Publisher,Eidos Interactive,Eidos Interactive,Konami Digital Entertainment,Konami Digital Entertainment,Konami Digital Entertainment
Critic_Score,84,91,87,70,71
Critic_Count,23,17,28,54,41
User_Score,8,9,8.5,6.9,6.9
User_Count,19,132,39,180,143
Developer,Io Interactive,Crystal Dynamics,KCEJ,Double Helix Games,Double Helix Games


In [8]:
df_submission.head()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Global_Sales
0,63.371815
1,63.371815
2,63.371815
3,63.371815
4,63.371815


In [9]:
df_train.loc[:, 'has_na_sales'] = df_train.loc[:, 'NA_Sales']  > 0
df_train.loc[:, 'has_eu_sales'] = df_train.loc[:, 'EU_Sales']  > 0
df_train.loc[:, 'has_jp_sales'] = df_train.loc[:, 'JP_Sales']  > 0
df_train.loc[:, 'has_other_sales'] = df_train.loc[:, 'Other_Sales']  > 0
# df_train.to_csv(datadir / 'country_train.csv', index=False)

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# 要検討
df_best_pred = pd.read_csv('../ckpt/187_name_count/187_name_count.csv')
df_test.loc[:, 'Global_Sales'] = df_best_pred.loc[:, 'Global_Sales']

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
train = df_train.copy()
test = df_test.copy()
train['is_test'] = False
test['is_test'] = True
df_whole = pd.concat([train, test])

In [12]:
train_others = {
    'main': df_train.copy(),
    'another': df_test.copy()
}
test_others = {
    'main': df_test.copy(),
    'another': df_train.copy(),
}

# Publisher

In [13]:
df_pub_platform = df_whole.groupby(['Platform', 'Publisher'])['Name'].count().reset_index()
df_platform_max = df_pub_platform.groupby('Platform')['Name'].max().reset_index()
df_platform_max = change_column_name(df_platform_max, 'Name', 'max_count')
df_pub_platform = pd.merge(df_pub_platform, df_platform_max, how='left', on='Platform')
df_pub_platform = df_pub_platform.loc[df_pub_platform.loc[:, 'Name'] == df_pub_platform.loc[:, 'max_count'], :]
df_pub_platform

,Platform,Publisher,Name,max_count
3,2600,Atari,54,54
26,3DO,Imageworks,1,1
27,3DO,Konami Digital Entertainment,1,1
28,3DO,Micro Cabin,1,1
80,3DS,Namco Bandai Games,88,88
122,DC,Sega,30,30
289,DS,Ubisoft,181,181
314,GB,Nintendo,61,61
389,GBA,THQ,110,110
418,GC,Electronic Arts,90,90


In [21]:
df_pub_train = df_train.groupby('Publisher')['Name'].count().reset_index()
df_pub_test = df_test.groupby('Publisher')['Name'].count().reset_index()
df_pub_train_target = df_train.groupby('Publisher')['Global_Sales'].agg(['mean', 'median', 'sum']).reset_index()
df_pub_test_target = df_test.groupby('Publisher')['Global_Sales'].agg(['mean', 'median', 'sum']).reset_index()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
df_pub_train = pd.merge(df_pub_train, df_pub_train_target, how='left', on='Publisher')
df_pub_test = pd.merge(df_pub_test, df_pub_test_target, how='left', on='Publisher')

In [23]:
df_pub_train.sort_values(by='Name', ascending=False).head(20)

,Publisher,Name,mean,median,sum
255,THQ,715,47.314685,26.0,33830
186,Nintendo,706,253.327195,89.0,178849
238,Sony Computer Entertainment,687,88.250364,33.0,60628
231,Sega,638,42.344828,19.0,27016
261,Take-Two Interactive,422,95.644550,29.0,40362
48,Capcom,386,51.787565,19.0,19990
26,Atari,367,42.713896,18.0,15676
263,Tecmo Koei,348,15.367816,8.0,5348
282,Warner Bros. Interactive Entertainment,235,64.561702,37.0,15172
244,Square Enix,234,61.649573,33.0,14426


In [24]:
df_pub_test.sort_values(by='Name', ascending=False).head(20)

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Publisher,Name,mean,median,sum
68,Electronic Arts,1356,61.568857,47.304988,83487.370573
13,Activision,985,56.693242,40.888164,55842.843305
169,Namco Bandai Games,939,30.911217,17.938834,29025.632328
256,Ubisoft,933,48.097815,31.033305,44875.261199
139,Konami Digital Entertainment,834,35.275498,21.102241,29419.765011
261,Unknown,201,14.643041,11.288448,2943.251329
159,Midway Games,198,31.029910,23.117619,6143.922174
67,Eidos Interactive,198,42.809722,23.169357,8476.324994
11,Acclaim Entertainment,184,25.813585,21.028287,4801.326807
40,Codemasters,150,39.249454,29.437860,5887.418055
